In [ ]:
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import geopy.distance
from geopy.distance import geodesic

def geo_to_row_col(lon, lat, transform):
    """Convert geographic coordinates to row and column indices."""
    col, row = ~transform * (lon, lat)  # The ~ operator inverts the affine transform
    return int(row), int(col)

# Read the DEM .tif file
with rasterio.open('???.tif') as src:
    dem_data = src.read(1)
    transform = src.transform

# Plot the DEM
plt.imshow(dem_data, cmap='terrain')
plt.colorbar(label='Elevation')
plt.title('DEM Data')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()

# Ask user for two points
lon1, lat1 = map(float, input("Enter the geographic coordinates of the first point (lon1, lat1): ").strip("() ").split(", "))
lon2, lat2 = map(float, input("Enter the geographic coordinates of the second point (lon2, lat2): ").strip("() ").split(", "))
#lon1, lat1 = 18.368978, 19.831981
#lon2, lat2 = 18.744875, 19.833513

# Convert geographic coordinates to row and column indices
row1, col1 = geo_to_row_col(lon1, lat1, transform)
row2, col2 = geo_to_row_col(lon2, lat2, transform)

# Generate profile between points
num_points = 1000  # Number of points in the profile
row_values = np.linspace(row1, row2, num_points).astype(int)
col_values = np.linspace(col1, col2, num_points).astype(int)
z_values = [dem_data[row, col] for row, col in zip(row_values, col_values)]

# Compute the total geodesic distance between start and end point
start_point = (lat1, lon1)
end_point = (lat2, lon2)
total_distance_km = geodesic(start_point, end_point).kilometers

# Generate distances for all points in the profile (linearly spaced)
distances_km = np.linspace(0, total_distance_km, num_points)

# Plot elevation profile against distances
plt.figure()
plt.plot(distances_km, z_values)
plt.title(f'Elevation profile between ({lon1}, {lat1}) and ({lon2}, {lat2})')
plt.xlabel('Distance (km)')
plt.ylabel('Elevation')

# Extend y-axis to 800 minimum (optional)
# plt.ylim(0, plt.ylim()[1])
# plt.ylim(0, 4000)
# plt.savefig('profile.ps', format='ps')  # Save the figure in PostScript format
plt.show()